In [1]:
import ikamoana
import parcels
import numpy as np
import xarray as xr
import ikamoana.SEAPODYM_functions_Parcels3 as sfp
import ikamoana

In [2]:
%load_ext line_profiler

## __Initialize Habitat__

In [3]:

xml_filepath="./data/feeding_habitat_data/po_2x30_historical_interim/skj_po_interim_2deg_configuration_file.xml"
fh = ikamoana.feedinghabitat.FeedingHabitat(xml_filepath)

In [4]:
fh_test_field_set = fh.computeFeedingHabitat(0)
#fh_test_field_set = fh_test_field_set.sel(time=fh_test_field_set.time[:1])
habitat_0 = fh_test_field_set['Feeding_Habitat_Cohort_0']

## __Add Ubathy__

In [5]:
# lmeso = lower meso
ubathy = fh.data_structure.variables_dictionary['forage_lmeso']

fh_test_field_set_to_merge = xr.Dataset(
    {'Ubathy':ubathy},
    coords=fh_test_field_set.coords,
)

final_test_field_set = fh_test_field_set.merge(fh_test_field_set_to_merge)
final_test_field_set

<xarray.Dataset>
Dimensions:                   (lon: 101, lat: 60, time: 384)
Coordinates:
  * lon                       (lon) float32 89.5 91.5 93.5 ... 285.5 287.5 289.5
  * lat                       (lat) float32 64.5 62.5 60.5 ... -49.5 -51.5 -53.5
  * time                      (time) datetime64[ns] 1979-01-15T12:00:00 ... 2...
Data variables:
    Feeding_Habitat_Cohort_0  (time, lat, lon) float32 nan nan nan ... nan nan
    Ubathy                    (time, lat, lon) float32 0.0 0.0 0.0 ... nan nan
Attributes: (12/14)
    time_start:  None
    time_end:    None
    lat_min:     None
    lat_max:     None
    lon_min:     None
    lon_max:     None
    ...          ...
    sigma_K:     3.00000257690523
    T_star_1:    31.09996321152416
    T_star_K:    26.04960048814059
    bT:          2.83057972721579
    gamma:       0.0001000000005808978
    o_star:      3.764163854561827

## __Convert DataSet to FieldSet__

In [6]:
variables = {
    'H': 'Feeding_Habitat_Cohort_0',
    'Ubathy' : 'Ubathy'}

dimensions = {
    'lon': 'lon',
    'lat': 'lat',
    'time': 'time'}

field_set_test = parcels.FieldSet.from_xarray_dataset(
    final_test_field_set.reindex(lat=list(reversed(final_test_field_set.lat))),
    variables=variables,
    dimensions=dimensions)

## __Create Landmask__

In [7]:
field_set_test.H.grid.lat

array([-53.5, -51.5, -49.5, -47.5, -45.5, -43.5, -41.5, -39.5, -37.5,
       -35.5, -33.5, -31.5, -29.5, -27.5, -25.5, -23.5, -21.5, -19.5,
       -17.5, -15.5, -13.5, -11.5,  -9.5,  -7.5,  -5.5,  -3.5,  -1.5,
         0.5,   2.5,   4.5,   6.5,   8.5,  10.5,  12.5,  14.5,  16.5,
        18.5,  20.5,  22.5,  24.5,  26.5,  28.5,  30.5,  32.5,  34.5,
        36.5,  38.5,  40.5,  42.5,  44.5,  46.5,  48.5,  50.5,  52.5,
        54.5,  56.5,  58.5,  60.5,  62.5,  64.5], dtype=float32)

In [8]:
landmask = sfp.Create_Landmask(field_set_test)
print("Landmask is :\n")
landmask.data

100% (101 of 101) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


Landmask is :



array([[[0., 0., 0., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

## __Compute Gradient__

In [9]:
gradient_dx_dy = sfp.getGradient(field=field_set_test.H, landmask=landmask)
print("\nDX is :\n",gradient_dx_dy[0].data[0])
print("\nDY is :\n",gradient_dx_dy[1].data[0])

Field for gradient calculation has time origin of 1979-01-15T12:00:00.000000000

DX is :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

DY is :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
gradient_dx_dy[0].data.dtype

dtype('float32')

## __Compute Taxis__

In [11]:
taxis_tx_ty = sfp.Create_SEAPODYM_Taxis_Fields(
    dHdx=gradient_dx_dy[0], dHdy=gradient_dx_dy[1],
    length_classes=fh.data_structure.species_dictionary['cohorts_mean_length'],
    start_age=0)

print("Taxis (tx) is :\n", taxis_tx_ty[0].data[0])
print("Taxis (ty) is :\n", taxis_tx_ty[1].data[0])

Taxis for 384 steps
Taxis (tx) is :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Taxis (ty) is :
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


---

In [12]:
xml_filepath = "./data/feeding_habitat_data/po_2x30_historical_interim/skj_po_interim_2deg_configuration_file.xml"

fh = ikamoana.feedinghabitat.FeedingHabitat(xml_filepath)
ikaf = ikamoana.ikamoanafields.IkamoanaFields(xml_filepath,xml_filepath)

## 1. Compute Taxis for Cohort number 0
### 1.1 Compute Feeding Habitat

In [13]:
landmask_2 = ikaf.landmask(habitat_0)
landmask_2.data.shape

(60, 101)

In [14]:
print(np.sum(landmask.data[0,:,:] == np.flip(landmask_2.data, axis=0)))
print(landmask_2.data.size)

6060
6060


### 1.2 Compute Gradient

In [15]:
grad_data = ikaf.gradient(habitat_0,landmask_2)

### 1.3 Compute Taxis

In [16]:
taxis_data = ikaf.taxis(grad_data[0], grad_data[1])
taxis_data[0]

<xarray.DataArray 'Tlon' (time: 384, lat: 60, lon: 101)>
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
...
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 89.5 91.5 93.5 95.5 97.5 ... 283.5 285.5 287.5 289.5
  * lat      (lat) float32 64.5 62.5 60.5 58.5 56.5 ... -47.5 -49.5 -51.5 -53.5
  * time     (time) datetime64[ns] 1979-01-15T12:00:00 ... 2010-12-16T12:00:00
Attributes:
    Cohort number:     0
    Age start (days):  0.0
    Age end (days):    30.0
    Length (cm):       3.0
    Weight (kg):       0.0003